# Подход 1: градиентный бустинг "в лоб"
Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше.

### Загружаем и разархивируем features.zip — архив с обучающей выборкой

In [ ]:
try:
    import wget
except:
    !pip install wget > nul
    import wget
import zipfile
zip_file = wget.download('https://d3c33hcgiwev3.cloudfront.net/_47ea8666d262362a0b41164cb497658e_features.zip?Expires=1608595200&Signature=fMbNpjE8hxxRyYp6V7E8cQlsDVzvN~~JLILkv8rEmDUL9xEQfUAJ-wa8AA560QrosdOtkkvJ6A-QE6Tn1pM5qTfv2kFTQpYblx2hoqH6VsO5vGZdhhhiUEoUO7Pe0Rewy8OvSzGEkSwL6b2av3Ar8bVxK~h-wXY7lE-1f2OK22A_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A','./features.zip')
with zipfile.ZipFile(zip_file,"r") as f:
  f.extractall()

In [ ]:
import pandas as pd
data_train = pd.read_csv('./features.csv', index_col='match_id')
data_train

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,...,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,...,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,NaN,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,...,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,NaN,173.0,-80.0,NaN,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,...,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,NaN,63.0,-82.0,NaN,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,...,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208.0,-75.0,NaN,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,...,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,NaN,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,2449,0,4,1974,3,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,7,2,616,535,0,0,2,10,1,3,879,1798,22,1,1,9,21,4,1835,1350,11,1,1,9,71,3,604,567,1,0,...,972,847,1,1,1,11,28,3,1583,1634,10,2,0,6,114.0,0.0,2.0,8.0,245.0,-86.0,211.0,5,2,1,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0,2032,0,1792,1975,48,63
114403,1450277704,0,43,4,1793,1416,17,0,1,5,26,3,764,800,0,1,0,9,4,3,1130,1585,20,1,1,14,29,4,1546,1166,8,1,0,6,110,3,892,541,1,0,...,1260,1008,4,1,1,12,98,5,1791,1186,3,1,0,8,174.0,1.0,9.0,4.0,139.0,-85.0,202.0,5,3,2,0,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0,1734,1,2038,6,63,3
114404,1450291848,1,98,4,1399,540,1,0,0,5,11,4,1448,1371,17,0,1,13,112,3,658,646,2,0,0,10,81,4,1710,1833,24,1,0,10,50,2,364,674,1,0,...,1778,1614,27,0,0,8,31,3,841,499,1,0,0,10,108.0,0.0,3.0,5.0,43.0,-83.0,181.0,2,4,4,2,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0,2906,0,1796,1846,51,63


### Загружаем и разархивируем features_test.zip — архив с тестовой выборкой

In [ ]:
zip_file_test = wget.download('https://d3c33hcgiwev3.cloudfront.net/_6866d458468373e164b1e859f3103826_features_test.zip?Expires=1608595200&Signature=OHYPh5u17GdTOxLav9LBwAfh66CMcAuzMd0u04vyGrD0hrMsvS2DrSNJEku5VJSC3PT5DaLKCXR1JCqijvOGleftRw8d6JonTab6K-SHIlqOGYDnYoGw5kgH7FxfdVw05F3Quo8n~XYreOkSyvijueAxsHl8rqEXCxwTQjz1o5c_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A','./features_test.zip')
with zipfile.ZipFile(zip_file_test,"r") as f_test:
  f_test.extractall()
data_test = pd.read_csv('./features_test.csv', index_col='match_id')
data_test

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,...,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,102,3,1183,963,9,0,2,9,52,5,2289,2043,20,2,0,6,53,3,917,1006,14,0,0,6,57,3,1344,933,3,0,...,2,1,0,9,29,3,1091,918,6,0,1,8,39,5,1674,1048,12,0,1,7,1.0,0.0,2.0,NaN,72.0,-82.0,NaN,2,2,3,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,6,4,1194,1386,21,0,0,7,65,4,1374,1064,13,0,1,8,92,2,774,737,0,1,0,8,43,5,2254,1787,24,0,...,19,0,0,14,99,3,1075,718,3,0,1,7,5,4,1130,994,1,1,0,10,84.0,1.0,9.0,NaN,138.0,-82.0,233.0,2,6,2,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,26,2,421,569,3,0,0,6,29,3,960,896,3,1,2,6,11,5,1877,1511,23,0,0,8,41,4,1581,1593,26,0,...,12,0,0,8,49,5,2434,1917,22,0,0,9,54,4,1372,2005,24,2,1,11,168.0,1.0,9.0,NaN,33.0,-88.0,NaN,5,4,3,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,91,2,672,901,6,0,0,8,74,5,2141,1626,28,0,0,7,52,4,1579,2169,28,2,0,7,2,5,1960,1668,27,0,...,22,0,0,7,62,2,536,692,0,1,1,6,49,4,1815,1424,18,0,0,5,55.0,0.0,3.0,NaN,208.0,-78.0,180.0,4,2,2,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,93,5,1703,964,10,0,1,6,30,3,1099,942,2,1,0,6,85,3,1285,1013,10,0,0,8,102,2,484,609,4,0,...,19,0,0,10,3,3,623,520,1,0,0,10,96,5,1884,1409,15,0,1,11,285.0,1.0,5.0,NaN,225.0,-76.0,NaN,1,3,2,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,1450212780,7,11,5,2054,1941,27,0,1,8,28,4,1460,936,7,0,1,8,89,3,1208,1579,22,0,1,13,20,2,464,816,1,0,1,10,86,3,576,932,3,1,...,19,4,0,9,74,4,2042,1287,12,0,0,5,37,3,1143,742,5,0,0,8,27.0,1.0,7.0,1.0,110.0,-84.0,180.0,5,3,4,1,8.0,253.0,-87.0,NaN,4,3,2,1,-33.0
114377,1450222875,1,3,3,748,605,1,0,0,12,22,3,1130,1385,16,1,0,9,60,3,779,660,0,0,0,8,11,4,1463,1362,18,0,0,8,8,4,1549,1479,24,0,...,18,0,0,7,31,3,633,500,0,0,0,10,72,4,1197,1245,18,0,1,8,147.0,1.0,5.0,4.0,NaN,-82.0,182.0,2,4,2,1,-1.0,133.0,-85.0,184.0,2,3,4,1,-18.0
114378,1450223593,1,85,2,575,499,0,0,0,8,102,3,697,686,3,0,0,8,75,5,2050,980,8,0,1,8,39,3,853,1014,13,0,1,6,47,4,1325,1390,21,0,...,2,1,0,10,98,5,1622,1684,22,1,0,10,34,5,2104,1963,25,1,0,8,4.0,1.0,7.0,3.0,NaN,-84.0,182.0,4,2,2,0,20.0,133.0,-88.0,239.0,4,4,4,0,-36.0


In [ ]:
train_Y=data_train['radiant_win']

Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [ ]:
columns_train_difference=data_train.columns.difference(data_test.columns.values.tolist()).tolist()
data_train.drop(columns_train_difference, axis=1, inplace=True)
data_train

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,...,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,...,23,0,0,6,37,3,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,0.0,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,...,2,2,0,12,7,2,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,0.0,173.0,-80.0,0.0,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,...,3,0,0,8,29,4,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,0.0,63.0,-82.0,0.0,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,...,11,0,0,7,102,2,674,537,1,0,0,7,20,2,510,499,0,0,0,7,0.0,0.0,0.0,0.0,208.0,-75.0,0.0,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,...,1,0,0,9,25,5,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,0.0,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,7,2,616,535,0,0,2,10,1,3,879,1798,22,1,1,9,21,4,1835,1350,11,1,1,9,71,3,604,567,1,0,...,18,0,1,9,3,3,972,847,1,1,1,11,28,3,1583,1634,10,2,0,6,114.0,0.0,2.0,8.0,245.0,-86.0,211.0,5,2,1,0,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,26,3,764,800,0,1,0,9,4,3,1130,1585,20,1,1,14,29,4,1546,1166,8,1,0,6,110,3,892,541,1,0,...,1,0,1,6,20,4,1260,1008,4,1,1,12,98,5,1791,1186,3,1,0,8,174.0,1.0,9.0,4.0,139.0,-85.0,202.0,5,3,2,0,-5.0,0.0,-82.0,0.0,4,3,2,0,-17.0
114404,1450291848,1,98,4,1399,540,1,0,0,5,11,4,1448,1371,17,0,1,13,112,3,658,646,2,0,0,10,81,4,1710,1833,24,1,0,10,50,2,364,674,1,0,...,16,1,0,8,59,4,1778,1614,27,0,0,8,31,3,841,499,1,0,0,10,108.0,0.0,3.0,5.0,43.0,-83.0,181.0,2,4,4,2,-32.0,249.0,-70.0,0.0,1,1,3,1,-15.0


2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [ ]:
train_size=len(data_train)
print(f"Количество строк: {train_size}" )
for col in data_train.columns.values.tolist():
    count=data_train[col].count()
    if count!=train_size:
        print(f"Признак: {col}, значений: {count}")

Количество строк: 97230
Признак: first_blood_time, значений: 77677
Признак: first_blood_team, значений: 77677
Признак: first_blood_player1, значений: 77677
Признак: first_blood_player2, значений: 53243
Признак: radiant_bottle_time, значений: 81539
Признак: radiant_courier_time, значений: 96538
Признак: radiant_flying_courier_time, значений: 69751
Признак: radiant_first_ward_time, значений: 95394
Признак: dire_bottle_time, значений: 81087
Признак: dire_courier_time, значений: 96554
Признак: dire_flying_courier_time, значений: 71132
Признак: dire_first_ward_time, значений: 95404


3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [ ]:
data_train.fillna(0, method=None, axis=1, inplace=True)

4. Какой столбец содержит целевую переменную? Запишите его название.

### Целевая переменная 'radiant_win'- 1, если победила команда Radiant, 0 — иначе

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [ ]:
import time
import datetime
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, random_state=42, shuffle=True)
n_estimators = {10, 20, 30, 100}

In [ ]:
for est in n_estimators:
  start_time = datetime.datetime.now()
  clf = GradientBoostingClassifier(n_estimators=est)
  scores = cross_val_score(clf, data_train, train_Y, scoring='roc_auc', cv=kf)
  print ('Time elapsed:', datetime.datetime.now() - start_time)
  mean_score = round(scores.mean()*100,2)
  print(f"Количество деревьев {est}, качество roc_auc на кросс-валидации по тренировочной выборке: {mean_score}%")

Time elapsed: 0:09:04.645642
Количество деревьев 100, качество roc_auc на кросс-валидации по тренировочной выборке: 70.62%
Time elapsed: 0:00:55.755712
Количество деревьев 10, качество roc_auc на кросс-валидации по тренировочной выборке: 66.49%
Time elapsed: 0:01:51.532591
Количество деревьев 20, качество roc_auc на кросс-валидации по тренировочной выборке: 68.25%
Time elapsed: 0:02:47.356322
Количество деревьев 30, качество roc_auc на кросс-валидации по тренировочной выборке: 69.0%


#ОТЧЕТ по подходу №1
## 1.Пропуски имеют следующие признаки:
* first_blood_time: игровое время первой крови
* first_blood_team: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
* first_blood_player1: игрок, причастный к событию
* first_blood_player2: второй игрок, причастный к событию
* radiant_bottle_time: время первого приобретения командой предмета "bottle"
* radiant_courier_time: время приобретения предмета "courier"
* radiant_flying_courier_time: время приобретения предмета "flying_courier
* radiant_first_ward_time: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
* dire_bottle_time: время первого приобретения командой предмета "bottle"
* dire_courier_time: время приобретения предмета "courier"
* dire_flying_courier_time: время приобретения предмета "flying_courier
* dire_first_ward_time: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
### Отсутствующие данные, скорее всего, свидетельствуют от том, что данные события не призошли в течение 5 минут игры.
## 2.'radiant_win'
## 3. C 30 деревьями кросс-валидация выполнялась ~2.30 мин.
## 4. В данном случае увеличивать деревья больше 30  имеет смысл при наличии достаточного времени или вычислительных мощностей. Максимальное количество деревьев можно подобрать с помощью GridSearchCV, но это довольно ресурсоемкий метод. При 100 деревьях качество увеличивается менее чем на 2%, а время подсчета почти в 4 раза. Для ускорения процесса при увеличении числа деревьев следует уменьшать глубину деревьев и/или использовать только часть признаков.


### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

**Важно:** не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться sklearn.preprocessing.StandartScaler.

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd

Воспользуемся StandardScaler

In [ ]:
data_train_sc=pd.DataFrame(data=StandardScaler().fit_transform(data_train))
data_train_sc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101
0,-2.544364,1.540688,-1.244228,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,0.453467,-0.332256,-0.625222,-0.255162,-0.052580,-0.533149,-0.582396,-1.756715,-0.728192,1.507230,1.308568,0.095346,-0.037204,-0.529081,-0.578154,-0.112095,-0.999319,-0.282744,-0.729830,-0.757388,-0.446505,-0.525343,-0.579877,-0.525183,1.606179,-0.277630,-0.771379,-0.935772,-0.669994,-0.524078,...,1.416964,-0.538253,-0.571065,-0.947053,-0.490513,-0.311205,-0.785580,-1.312874,-0.905651,-0.53886,-0.567540,-0.131690,0.950987,-0.316231,-0.398260,-0.192406,-0.794164,0.976866,-0.565289,0.278256,-0.849902,1.182703,1.681514,-0.788219,0.378538,-0.029781,0.876751,-0.514555,-1.136970,-0.532046,-0.987486,1.066448,-0.041743,-0.262922,0.640648,0.018054,0.562864,-0.551154,1.846004,-1.121494
1,-2.540452,-0.927798,-0.292258,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,-0.097431,0.578881,0.732454,-0.250795,-0.052580,-0.533149,1.019572,-0.524101,0.438857,0.591580,0.582307,0.884835,0.856569,1.014674,-0.578154,-0.521910,-0.478969,-0.282744,-0.889558,-1.001642,-0.334372,-0.525343,-0.579877,-0.525183,-0.816535,-1.190175,-1.344598,-1.198371,-1.005251,-0.524078,...,-0.942193,2.506742,-0.571065,1.517711,-1.414707,-1.232681,-1.337375,-1.127777,-1.131068,-0.53886,-0.567540,-0.543653,-1.262018,0.599652,0.531154,1.183531,1.570177,-0.540155,-0.565289,0.278256,-0.287631,1.182703,1.050584,-0.788219,0.912215,-0.029781,-1.525446,-0.514555,-2.869460,-0.532046,-0.987486,-0.338591,0.578946,-0.262922,0.379585,1.066668,0.562864,0.678170,0.437788,0.043947
2,-2.539231,1.540688,-0.568637,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,1.402237,-0.332256,0.224676,-0.726779,-0.498410,-0.533149,-0.582396,-0.934972,-1.004599,-0.324070,0.205660,-0.419251,-1.154419,1.014674,-0.578154,-0.931725,-0.785057,1.547927,2.179771,2.251377,1.796163,1.030097,1.043031,-0.937075,-1.491215,-0.277630,0.427498,1.202532,1.900308,-0.524078,...,-0.829853,-0.538253,-0.571065,-0.125465,-0.736965,0.610270,0.268676,-0.940451,-0.680234,-0.53886,-0.567540,-0.543653,0.828042,-0.316231,0.313323,0.939019,1.682765,-0.540155,-0.565289,-0.543829,1.746117,-0.845521,-0.211277,-0.788219,-0.593027,-0.146503,-1.525446,-0.514555,1.461765,-0.532046,0.391203,-0.823968,-0.824352,0.158654,0.640648,0.018054,0.562864,0.678170,0.437788,0.490286
3,-2.532622,-0.575157,-0.691471,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,-0.678935,-1.243393,-1.170813,-1.242065,-1.055698,-0.533149,-0.582396,-0.934972,0.684551,1.507230,1.539241,0.086550,0.521404,-0.529081,-0.578154,-0.931725,-0.478969,-1.198079,-1.031136,-1.294746,-1.119305,-0.525343,-0.579877,-0.937075,-0.356526,-0.277630,-0.807544,-0.015573,0.224024,-0.524078,...,0.068874,-0.538253,-0.571065,-0.536259,1.511906,-1.232681,-0.909825,-1.230361,-1.018359,-0.53886,-0.567540,-0.543653,-1.016129,-1.232114,-1.211497,-1.312717,-1.131927,-0.540155,-0.565289,-0.543829,-0.933645,-0.845521,-1.157673,-0.788219,1.391156,0.262024,-1.525446,-1.549070,-0.270725,-0.532046,-0.987486,-0.594053,0.241615,-0.022021,0.269135,-1.554868,0.562864,-0.551154,-0.970428,0.837439
4,-2.529221,1.540688,-1.182811,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,-0.770751,-1.243393,-1.008757,-1.213680,-1.167155,-0.533149,1.019572,-0.524101,-0.697480,-0.324070,-0.538623,-0.379666,-1.154419,1.014674,-0.578154,-0.112095,0.592339,-0.282744,-0.424893,0.792515,0.674829,1.030097,-0.579877,1.122386,1.238171,0.634915,0.584817,-0.445882,-0.334737,-0.524078,...,-1.054534,-0.538253,-0.571065,0.285329,-0.860191,1.531746,1.174935,0.415438,0.897683,-0.53886,-0.567540,-0.131690,-1.384963,0.599652,-0.106003,-0.412467,-0.456401,-0.540155,1.085005,-0.543829,-1.184873,1.182703,0.735119,-0.788219,0.816427,-0.088142,0.2565

In [ ]:
%%time
lr = LogisticRegression(n_jobs = -1);
grid = {'C': [0.001, 0.01, 0.05 ,0.1, 1, 5]} 
cv = KFold(n_splits=5, random_state=42, shuffle=True) #Кроссвалидация
gs = GridSearchCV(lr, grid, scoring='roc_auc', cv=cv, verbose=0)
gs.fit(data_train_sc, train_Y)
print('Лучший результат:', round(gs.best_score_*100,2))
print('Лучшие параметры:', gs.best_params_)

Лучший результат: 71.65
Лучшие параметры: {'C': 0.01}
CPU times: user 3.79 s, sys: 4.1 s, total: 7.89 s
Wall time: 2min 58s



2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [ ]:
cat_features = ['r%s_hero' % i for i in range(1, 6)]+['d%s_hero' % i for i in range(1, 6)]#колонки героев
cat_features.append('lobby_type')
data_train_new = data_train.drop(cat_features, axis=1)
data_train_new_sc=pd.DataFrame(data=StandardScaler().fit_transform(data_train_new))
data_train_new_sc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
0,-2.544364,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,-0.332256,-0.625222,-0.255162,-0.052580,-0.533149,-0.582396,-1.756715,1.507230,1.308568,0.095346,-0.037204,-0.529081,-0.578154,-0.112095,-0.282744,-0.729830,-0.757388,-0.446505,-0.525343,-0.579877,-0.525183,-0.277630,-0.771379,-0.935772,-0.669994,-0.524078,1.052453,1.115474,-0.417344,-0.322641,-0.337650,0.082842,...,1.558823,0.961587,1.416964,-0.538253,-0.571065,-0.947053,-0.311205,-0.785580,-1.312874,-0.905651,-0.53886,-0.567540,-0.131690,-0.316231,-0.398260,-0.192406,-0.794164,0.976866,-0.565289,0.278256,-0.849902,1.182703,1.681514,-0.788219,0.378538,-0.029781,0.876751,-0.514555,-1.136970,-0.532046,-0.987486,1.066448,-0.041743,-0.262922,0.640648,0.018054,0.562864,-0.551154,1.846004,-1.121494
1,-2.540452,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,0.578881,0.732454,-0.250795,-0.052580,-0.533149,1.019572,-0.524101,0.591580,0.582307,0.884835,0.856569,1.014674,-0.578154,-0.521910,-0.282744,-0.889558,-1.001642,-0.334372,-0.525343,-0.579877,-0.525183,-1.190175,-1.344598,-1.198371,-1.005251,-0.524078,-0.573665,-1.352836,1.392776,1.286498,0.507465,0.526938,...,-0.859912,0.548265,-0.942193,2.506742,-0.571065,1.517711,-1.232681,-1.337375,-1.127777,-1.131068,-0.53886,-0.567540,-0.543653,0.599652,0.531154,1.183531,1.570177,-0.540155,-0.565289,0.278256,-0.287631,1.182703,1.050584,-0.788219,0.912215,-0.029781,-1.525446,-0.514555,-2.869460,-0.532046,-0.987486,-0.338591,0.578946,-0.262922,0.379585,1.066668,0.562864,0.678170,0.437788,0.043947
2,-2.539231,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,-0.332256,0.224676,-0.726779,-0.498410,-0.533149,-0.582396,-0.934972,-0.324070,0.205660,-0.419251,-1.154419,1.014674,-0.578154,-0.931725,1.547927,2.179771,2.251377,1.796163,1.030097,1.043031,-0.937075,-0.277630,0.427498,1.202532,1.900308,-0.524078,-0.573665,0.292704,1.392776,1.901968,1.910183,0.860010,...,-0.796356,-1.073911,-0.829853,-0.538253,-0.571065,-0.125465,0.610270,0.268676,-0.940451,-0.680234,-0.53886,-0.567540,-0.543653,-0.316231,0.313323,0.939019,1.682765,-0.540155,-0.565289,-0.543829,1.746117,-0.845521,-0.211277,-0.788219,-0.593027,-0.146503,-1.525446,-0.514555,1.461765,-0.532046,0.391203,-0.823968,-0.824352,0.158654,0.640648,0.018054,0.562864,0.678170,0.437788,0.490286
3,-2.532622,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,-1.243393,-1.170813,-1.242065,-1.055698,-0.533149,-0.582396,-0.934972,1.507230,1.539241,0.086550,0.521404,-0.529081,-0.578154,-0.931725,-1.198079,-1.031136,-1.294746,-1.119305,-0.525343,-0.579877,-0.937075,-0.277630,-0.807544,-0.015573,0.224024,-0.524078,-0.573665,-0.941451,1.392776,1.140213,0.050057,0.637962,...,0.885136,-0.116160,0.068874,-0.538253,-0.571065,-0.536259,-1.232681,-0.909825,-1.230361,-1.018359,-0.53886,-0.567540,-0.543653,-1.232114,-1.211497,-1.312717,-1.131927,-0.540155,-0.565289,-0.543829,-0.933645,-0.845521,-1.157673,-0.788219,1.391156,0.262024,-1.525446,-1.549070,-0.270725,-0.532046,-0.987486,-0.594053,0.241615,-0.022021,0.269135,-1.554868,0.562864,-0.551154,-0.970428,0.837439
4,-2.529221,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,-1.243393,-1.008757,-1.213680,-1.167155,-0.533149,1.019572,-0.524101,-0.324070,-0.538623,-0.379666,-1.154419,1.014674,-0.578154,-0.112095,-0.282744,-0.424893,0.792515,0.674829,1.030097,-0.579877,1.122386,0.634915,0.584817,-0.445882,-0.334737,-0.524078,-0.573665,-0.118681,-0.417344,-0.954166,-1.230685,-1.138424,...,-0.251596,-1.033912,-1.054534,-0.538253,-0.571065,0.285329,1.531746,1.174935,0.415438,0.897683,-0.53886,-0.567540,-0.131690,0.599652,-0.106003,-0.412467,-0.456401,-0.540155,1.085005,-0.543829,-1.184873,1.182703,0.735119,-0.788219,0.816427,-0.088142,0.256512,-1.031

In [ ]:
%%time
gs = GridSearchCV(lr, grid, scoring='roc_auc', cv=cv, verbose=0)
gs.fit(data_train_new_sc, train_Y)
print('Лучший результат:', round(gs.best_score_*100,2))
print('Лучшие параметры:', gs.best_params_)

Лучший результат: 71.66
Лучшие параметры: {'C': 0.01}
CPU times: user 3.55 s, sys: 3.89 s, total: 7.44 s
Wall time: 2min 30s


3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [ ]:
cat_features.remove('lobby_type')
N_hero=pd.Series(data_train[cat_features].values.flatten()).drop_duplicates().shape[0]
print(f'В игре: {N_hero} героев')

В игре: 108 героев


4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [ ]:
#Функция ix устарела, воспользуемся loc

import numpy as np

X_pick = np.zeros((data_train.shape[0], N_hero))
 
for i, match_id in enumerate(data_train.index):
    for p in range(5):
        X_pick[i, data_train.loc[match_id, 'r%d_hero' % (p+1)]-N_hero] = 1
        X_pick[i, data_train.loc[match_id, 'd%d_hero' % (p+1)]-N_hero] = -1

In [ ]:
#Добавляем "Мешoк слов" к отмасштабированной выбрке числовых признаков, без категориальных 
full_data = np.hstack([np.array(data_train_new_sc), X_pick])
full_data.shape

(97230, 199)

5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [ ]:
%%time
lr = LogisticRegression(n_jobs = -1);
grid = {'C': np.linspace(0.03, 0.1, num=8)} 
cv = KFold(n_splits=5, random_state=42, shuffle=True) #Кроссвалидация
gs = GridSearchCV(lr, grid, scoring='roc_auc', cv=cv, verbose=0)
gs.fit(full_data, train_Y)
print('Лучший результат:', round(gs.best_score_*100,2))
print('Лучшие параметры:', gs.best_params_)

Лучший результат: 74.99
Лучшие параметры: {'C': 0.05}
CPU times: user 5.83 s, sys: 6.38 s, total: 12.2 s
Wall time: 18min 2s


6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [ ]:
#Заменяем пропуски на "0"
data_test.fillna(0, method=None, axis=1, inplace=True)
#Выкидываем из тестовой выборки категориальные признаки
cat_features = ['r%s_hero' % i for i in range(1, 6)]+['d%s_hero' % i for i in range(1, 6)]#колонки героев
cat_features.append('lobby_type')
data_test_new = data_test.drop(cat_features, axis=1)
# Масштабируем признаки
data_test_new_sc=pd.DataFrame(data=StandardScaler().fit_transform(data_test_new))
data_test_new_sc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
0,-2.514875,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,0.003979,-0.304443,-0.154739,-0.530684,2.625495,0.305627,1.497577,1.986390,2.078705,1.089605,2.548233,-0.576696,-0.935309,-0.337928,-0.451603,-0.179721,0.436965,-0.524238,-0.574233,-0.931472,-0.321391,0.341913,-0.315164,-0.766382,-0.520831,-0.574775,0.286089,-0.458720,-0.207894,-0.433700,-0.689819,...,-0.641160,-0.195355,-0.932791,0.961231,-0.564979,0.274323,-0.346328,-0.142785,-0.376866,-0.448234,-0.544256,1.101032,-0.130614,1.520805,0.913823,-0.090599,0.219285,-0.541959,1.076903,-0.537347,-0.918858,-0.842517,-0.520393,-0.784550,-0.455346,-0.145679,-1.519287,-0.520411,-1.125347,0.696599,-0.992369,0.450016,1.908352,-0.400363,1.149479,0.017743,0.554260,-0.571552,-0.983356,3.200079
1,-2.513895,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,0.023887,0.635111,1.185010,-0.530684,-0.578521,-0.525490,0.566666,0.343576,-0.076035,0.303071,-0.526977,1.019230,-0.105828,-1.270990,-0.710585,-0.776060,-1.130442,1.021658,-0.574233,-0.112267,1.557942,1.996586,1.583373,1.601574,-0.520831,-0.574775,-0.943164,-0.458720,-0.585577,-0.388057,0.530232,...,0.520876,0.632550,0.976367,-0.542098,-0.564979,2.351215,-0.346328,-0.172034,-0.824390,-0.786121,-0.544256,1.101032,-0.541069,0.583595,-0.082303,-0.210953,-1.023425,0.972723,-0.568531,0.686630,0.075106,1.186920,1.687574,-0.784550,0.437200,-0.145679,0.768520,-0.520411,2.328613,-0.531512,1.788142,-0.574649,0.836945,1.723080,-1.597294,0.017743,-1.177689,-0.571552,0.433564,0.600867
2,-2.512377,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,-1.375081,-1.179583,-0.824614,-0.530684,-0.578521,-0.941049,-0.364246,-0.399730,-0.445796,-0.820550,1.010628,2.615155,-0.935309,1.528198,1.287019,0.939801,1.444585,-0.524238,-0.574233,-0.112267,0.618275,0.772855,1.152090,1.827094,-0.520831,-0.574775,-1.762666,-0.458720,-0.979220,-1.029242,-1.133474,...,-0.419218,-0.179860,0.190243,-0.542098,-0.564979,-0.141055,1.524094,2.312262,1.858517,1.353830,-0.544256,-0.559701,0.279842,0.583595,0.360827,2.042341,1.574968,2.487405,1.076903,1.094622,1.081045,1.186920,1.687574,-0.784550,-0.982759,-0.499916,-1.519287,1.048954,0.601633,0.696599,0.397887,-0.399706,-0.817633,-0.466721,0.281014,-1.038655,-0.311714,0.668978,-0.983356,-0.673257
3,-2.508381,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,-0.920823,-0.442155,-0.489676,-0.530684,-0.578521,-0.109932,1.497577,1.720667,1.160904,1.988501,-0.526977,-0.576696,-0.520569,0.595135,0.747322,2.398504,2.004373,2.567553,-0.574233,-0.521870,1.557942,1.461999,1.318823,1.939854,-0.520831,-0.574775,-0.123662,0.463282,0.027937,-0.966210,-0.689819,...,0.201497,0.557286,1.313277,-0.542098,-0.564979,-0.556433,-1.281539,-1.157343,-0.882568,-1.124009,0.987059,1.101032,-0.951524,0.583595,1.172010,0.747421,0.897126,-0.541959,-0.568531,-1.353332,-0.272183,-0.842517,-0.204970,-0.784550,1.383840,0.090479,0.248117,0.525832,-1.125347,-0.531512,1.788142,-1.074486,-1.034626,-0.599436,0.523376,0.017743,0.554260,-0.571552,0.433564,-0.469397
4,-2.507087,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,0.945070,-0.302222,-0.043093,-0.530684,1.023487,-0.941049,-0.364246,-0.150166,-0.344552,-0.932912,1.010628,-0.576696,-0.935309,-0.337928,0.214869,-0.164203,-0.010865,-0.524238,-0.574233,-0.112267,-1.261058,-1.221843,-1.035452,-0.653622,-0.520831,-0.574775,-0.123662,0.463282,1.833017,1.155134,0.530232,...,-0.110665,0.654686,0.976367,-0.542098,-0.564979,0.689702,-0.346328,-0.998304,-1.267439,-1.011380,-0.544256,-0.559701,0.690297,1.520805,1.298357,0.713990,0.558206,-0.541959,1.076903,1.094622,2.482176,1.186920,0.425878,-0.784550,1.613738,0.208558,-1.519287,-1.043533,-0.261857,-0.5

In [ ]:
# Составляем "Мешок слов"
X_pick_test = np.zeros((data_test.shape[0], N_hero))
 
for i, match_id in enumerate(data_test.index):
    for p in range(5):
        X_pick[i, data_test.loc[match_id, 'r%d_hero' % (p+1)]-N_hero] = 1
        X_pick[i, data_test.loc[match_id, 'd%d_hero' % (p+1)]-N_hero] = -1
#Добавляем "Мешoк слов" к отмасштабированной выбрке числовых признаков ТЕСТОВОГО датасета
full_data_test = np.hstack([np.array(data_test_new_sc), X_pick_test])
full_data_test.shape

(17177, 199)

In [ ]:
#Обучаем модель с наилучшими параметрами по тренировочной выборке
start_time = datetime.datetime.now()
final_model = LogisticRegression(C = 0.05, n_jobs = -1).fit(full_data, train_Y)
#Предсказывает вероятность на тестовой выборке
y_pred = final_model.predict_proba(full_data_test)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:00:34.747701


In [ ]:
#Проверка неконстантности модели
print(y_pred)
print("\nMинимальное значение прогноза: ", y_pred[:, 1].min())
print("\nМаксимальное значение прогноза: ", y_pred[:, 1].max() )

[[0.48498959 0.51501041]
 [0.33889545 0.66110455]
 [0.6797318  0.3202682 ]
 ...
 [0.77309387 0.22690613]
 [0.63890743 0.36109257]
 [0.46961339 0.53038661]]

Mинимальное значение прогноза:  0.005744518567402761

Максимальное значение прогноза:  0.9937054595749831


#**Отчет по подходу 2: логистическая регрессия**
##1. Качество на кроссвалидации (без дополнительной обработки признаков) у логистической регрессии над всеми исходными признаками - 71.65%, что выше чем 70.62% у градиентного бустинга. Разницу можно объяснить особенностями  исходных данных.
## Скорость деревьев заметно падает с увеличением их числа. На финальном тесте видно, что логистическая регрессия работает значительно быстрее.
##2. При удалении категориальных признаков качество модели возросло на 0,01% и составило 71,66%. Линейные модели не предназначены для работы с категориальными признаками, эти признаки сорее всего просто вносят дополнительный шум.
##3. По моим расчетам, в игре 108 героев.
##4. При добавлении "мешка слов" по героям, качество улучшается и составляет 74.99%. Это связано с переводом категориальных признаков в числовые на работу с которыми и рассчитана линейная регрессия.
##5. Минимальное значение прогноза -  0.005744518567402761
## Максимальное значение прогноза - 0.9937054595749831
